### [PCCP 기출문제] 1번 / 동영상 재생기

- 당신은 동영상 재생기를 만들고 있습니다. 당신의 동영상 재생기는 10초 전으로 이동, 10초 후로 이동, 오프닝 건너뛰기 3가지 기능을 지원합니다. 각 기능이 수행하는 작업은 다음과 같습니다.
    - 10초 전으로 이동: 사용자가 "prev" 명령을 입력할 경우 동영상의 재생 위치를 현재 위치에서 10초 전으로 이동합니다. 현재 위치가 10초 미만인 경우 영상의 처음 위치로 이동합니다. 영상의 처음 위치는 0분 0초입니다.
    - 10초 후로 이동: 사용자가 "next" 명령을 입력할 경우 동영상의 재생 위치를 현재 위치에서 10초 후로 이동합니다. 동영상의 남은 시간이 10초 미만일 경우 영상의 마지막 위치로 이동합니다. 영상의 마지막 위치는 동영상의 길이와 같습니다.
    - 오프닝 건너뛰기: 현재 재생 위치가 오프닝 구간(`op_start` ≤ 현재 재생 위치 ≤ `op_end`)인 경우 자동으로 오프닝이 끝나는 위치로 이동합니다.

- 동영상의 길이를 나타내는 문자열 `video_len`, 기능이 수행되기 직전의 재생위치를 나타내는 문자열 `pos`, 오프닝 시작 시각을 나타내는 문자열 `op_start`, 오프닝이 끝나는 시각을 나타내는 문자열 `op_end`, 사용자의 입력을 나타내는 1차원 문자열 배열 `commands`가 매개변수로 주어집니다. 이때 사용자의 입력이 모두 끝난 후 동영상의 위치를 "`mm`:`ss`" 형식으로 return 하도록 solution 함수를 완성해 주세요.

#### 제한사항

- `video_len`의 길이 = `pos`의 길이 = `op_start`의 길이 = `op_end`의 길이 = 5
    - `video_len`, `pos`, `op_start`, `op_end`는 "`mm`:`ss`" 형식으로 `mm`분 `ss`초를 나타냅니다.
    - 0 ≤ `mm` ≤ 59
    - 0 ≤ `ss` ≤ 59
    - 분, 초가 한 자리일 경우 0을 붙여 두 자리로 나타냅니다.
    - 비디오의 현재 위치 혹은 오프닝이 끝나는 시각이 동영상의 범위 밖인 경우는 주어지지 않습니다.
    - 오프닝이 시작하는 시각은 항상 오프닝이 끝나는 시각보다 전입니다.

- 1 ≤ `commands`의 길이 ≤ 100
    - `commands`의 원소는 "prev" 혹은 "next"입니다.
    - "prev"는 10초 전으로 이동하는 명령입니다.
    - "next"는 10초 후로 이동하는 명령입니다.

#### 입출력 예

| video_len | pos    | op_start | op_end | commands           | result |
|-----------|--------|----------|--------|--------------------|--------|
| "34:33"   | "13:00"| "00:55"  | "02:55"| ["next", "prev"]   | "13:00"|
| "10:55"   | "00:05"| "00:15"  | "06:55"| ["prev", "next", "next"] | "06:55"|
| "07:22"   | "04:05"| "00:15"  | "04:07"| ["next"]           | "04:17"|

In [13]:
def check_op(pos, op_start, op_end):
    if pos in range(op_start, op_end+1):
        pos = op_end
    return pos

def solution(video_len, pos, op_start, op_end, commands):
    answer = ''
    video_len = int(video_len[-2:]) + int(video_len[:2])*60
    pos = int(pos[-2:]) + int(pos[:2])*60
    op_start = int(op_start[-2:]) + int(op_start[:2])*60
    op_end = int(op_end[-2:]) + int(op_end[:2])*60
    
    pos = check_op(pos, op_start, op_end)
    
    for command in commands:
        if command == "next":
            pos += 10
            if pos > video_len:
                pos = video_len
            pos = check_op(pos, op_start, op_end)
        elif command == "prev":
            pos -= 10
            if pos < 0:
                pos = 0
            pos = check_op(pos, op_start, op_end)
    
    minute = pos//60 if len(str(pos//60)) == 2 else '0'+str(pos//60)
    second = pos - (pos//60)*60 if len(str(pos - (pos//60)*60)) == 2 else '0'+str(pos - (pos//60)*60)
    
    answer = f'{minute}:{second}'
    
    return answer

print(solution("34:33", "13:00", "00:55", "02:55", ["next", "prev"]))           # "13:00"
print(solution("10:55", "00:05", "00:15", "06:55", ["prev", "next", "next"]))   # "06:55"
print(solution("07:22", "04:05", "00:15", "04:07", ["next"]))                   # "04:17"

13:00
06:55
04:17


In [14]:
# 코드 구조 개선
def time_to_seconds(time_str):
    """시간 문자열을 초 단위로 변환."""
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

def seconds_to_time(seconds):
    """초 단위를 'mm:ss' 형식의 문자열로 변환."""
    minutes = seconds // 60
    secs = seconds % 60
    return f"{minutes:02}:{secs:02}"

def skip_opening(current_pos, op_start, op_end):
    """현재 위치가 오프닝 구간 내에 있다면 오프닝 종료 시점으로 이동."""
    if op_start <= current_pos <= op_end:
        return op_end
    return current_pos

def solution(video_len, pos, op_start, op_end, commands):
    # 시간 문자열을 초 단위로 변환
    video_len_sec = time_to_seconds(video_len)
    current_pos = time_to_seconds(pos)
    op_start_sec = time_to_seconds(op_start)
    op_end_sec = time_to_seconds(op_end)
    
    # 초기 위치가 오프닝 구간 내에 있는지 확인
    current_pos = skip_opening(current_pos, op_start_sec, op_end_sec)
    
    for command in commands:
        if command == "next":
            current_pos += 10
            if current_pos > video_len_sec:
                current_pos = video_len_sec
        elif command == "prev":
            current_pos -= 10
            if current_pos < 0:
                current_pos = 0
        # 명령어 실행 후 오프닝 구간 검사
        current_pos = skip_opening(current_pos, op_start_sec, op_end_sec)
    
    # 최종 위치를 'mm:ss' 형식으로 변환하여 반환
    return seconds_to_time(current_pos)

# 테스트 케이스
print(solution("34:33", "13:00", "00:55", "02:55", ["next", "prev"]))           # "13:00"
print(solution("10:55", "00:05", "00:15", "06:55", ["prev", "next", "next"]))   # "06:55"
print(solution("07:22", "04:05", "00:15", "04:07", ["next"]))                   # "04:17"

13:00
06:55
04:17


In [15]:
# 다른 풀이법
def time_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

def seconds_to_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    return f"{minutes:02d}:{seconds:02d}"

def solution(video_len, pos, op_start, op_end, commands):
    video_len = time_to_seconds(video_len)
    current_pos = time_to_seconds(pos)
    op_start = time_to_seconds(op_start)
    op_end = time_to_seconds(op_end)

    for command in commands:
        if op_start <= current_pos <= op_end:
            current_pos = op_end

        if command == "prev":
            current_pos = max(0, current_pos - 10)
        elif command == "next":
            current_pos = min(video_len, current_pos + 10)

    # 마지막 명령 실행 후 한 번 더 오프닝 체크
    if op_start <= current_pos <= op_end:
        current_pos = op_end

    return seconds_to_time(current_pos)

print(solution("34:33", "13:00", "00:55", "02:55", ["next", "prev"]))           # "13:00"
print(solution("10:55", "00:05", "00:15", "06:55", ["prev", "next", "next"]))   # "06:55"
print(solution("07:22", "04:05", "00:15", "04:07", ["next"]))                   # "04:17"

13:00
06:55
04:17


### [PCCE 기출문제] 9번 / 지폐 접기

- 민수는 다양한 지폐를 수집하는 취미를 가지고 있습니다. 지폐마다 크기가 달라 지갑에 넣으려면 여러 번 접어서 넣어야 합니다. 예를 들어 지갑의 크기가 30 * 15이고 지폐의 크기가 26 * 17이라면 한번 반으로 접어 13 * 17 크기로 만든 뒤 90도 돌려서 지갑에 넣을 수 있습니다. 지폐를 접을 때는 다음과 같은 규칙을 지킵니다.

    - 지폐를 접을 때는 항상 길이가 긴 쪽을 반으로 접습니다.

    - 접기 전 길이가 홀수였다면 접은 후 소수점 이하는 버립니다.
    
    - 접힌 지폐를 그대로 또는 90도 돌려서 지갑에 넣을 수 있다면 그만 접습니다.

- 지갑의 가로, 세로 크기를 담은 정수 리스트 `wallet`과 지폐의 가로, 세로 크기를 담은 정수 리스트 `bill`가 주어질 때, 지갑에 넣기 위해서 지폐를 최소 몇 번 접어야 하는지 return하도록 solution함수를 완성해 주세요.

- 지폐를 지갑에 넣기 위해 접어야 하는 최소 횟수를 구하는 과정은 다음과 같습니다.

```plaintext
1. 지폐를 접은 횟수를 저장할 정수 변수 answer를 만들고 0을 저장합니다.
2. 반복문을 이용해 bill의 작은 값이 wallet의 작은 값 보다 크거나 bill의 큰 값이 wallet의 큰 값 보다 큰 동안 아래 과정을 반복합니다.
    2-1. bill[0]이 bill[1]보다 크다면
        bill[0]을 2로 나누고 나머지는 버립니다.
    2-2. 그렇지 않다면
        bill[1]을 2로 나누고 나머지는 버립니다.
    2-3. answer을 1 증가시킵니다.
3. answer을 return합니다.
```

- 위의 의사코드와 작동방식이 다른 코드를 작성해도 상관없습니다.

#### 제한사항

- `wallet`의 길이 = `bill`의 길이 = 2

- 10 ≤ `wallet[0]`, `wallet[1]` ≤ 100

- 10 ≤ `bill[0]`, `bill[1]` ≤ 2,000

#### 입출력 예
| wallet | bill | result |
|--------|------|--------|
| [30, 15] | [26, 17] | 1 |
| [50, 50] | [100, 241] | 4 |

In [1]:
def solution(wallet, bill):
    answer = 0
    
    while min(bill) > min(wallet) or max(bill) > max(wallet):
        if bill[0] > bill[1]:
            bill[0] //= 2
        else:
            bill[1] //= 2
        
        answer += 1

    return answer

print(solution([30, 15], [26, 17]))
print(solution([50, 50], [100, 241]))

1
4


In [2]:
# 다른 풀이법
def solution(wallet, bill):
    wallet, bill = sorted(wallet), sorted(bill)
    cnt = 0
    while wallet[0] < bill[0] or wallet[1] < bill[1]:
        bill[-1] //= 2
        bill = sorted(bill)
        cnt += 1

    return cnt

print(solution([30, 15], [26, 17]))
print(solution([50, 50], [100, 241]))

1
4
